In [1]:
%matplotlib inline


LDA Model
=========

Introduces Gensim's LDA model and demonstrates its use on the NIPS corpus.


In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

The purpose of this tutorial is to demonstrate training an LDA model and
obtaining good results.

In this tutorial we will:

* Load data.
* Pre-process data.
* Transform documents to a vectorized form.
* Train an LDA model.

This tutorial will **not**:

* Explain how Latent Dirichlet Allocation works
* Explain how the LDA model performs inference
* Teach you how to use Gensim's LDA implementation in its entirety

If you are not familiar with the LDA model or how to use it in Gensim, I
suggest you read up on that before continuing with this tutorial. Basic
understanding of the LDA model should suffice. Examples:

* `Introduction to Latent Dirichlet Allocation <http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation>`_
* Gensim tutorial: `sphx_glr_auto_examples_core_run_topics_and_transformations.py`
* Gensim's LDA model API docs: :py:class:`gensim.models.LdaModel`

I would also encourage you to consider each step when applying the model to
your data, instead of just blindly applying my solution. The different steps
will depend on your data and possibly your goal with the model.

Data
----

I have used a corpus of NIPS papers in this tutorial, but if you're following
this tutorial just to learn about LDA I encourage you to consider picking a
corpus on a subject that you are familiar with. Qualitatively evaluating the
output of an LDA model is challenging and can require you to understand the
subject matter of your corpus (depending on your goal with the model).

NIPS (Neural Information Processing Systems) is a machine learning conference
so the subject matter should be well suited for most of the target audience
of this tutorial.  You can download the original data from Sam Roweis'
`website <http://www.cs.nyu.edu/~roweis/data.html>`_.  The code below will
also do that for you.

.. Important::
    The corpus contains 1740 documents, and not particularly long ones.
    So keep in mind that this tutorial is not geared towards efficiency, and be
    careful before applying the code to a large dataset.




In [ ]:
import io
import os.path
import re
import tarfile

import smart_open

def extract_documents(url='https://cs.nyu.edu/~roweis/data/nips12raw_str602.tgz'):
    fname = url.split('/')[-1]
    
    # Download the file to local storage first.
    # We can't read it on the fly because of 
    # https://github.com/RaRe-Technologies/smart_open/issues/331
    if not os.path.isfile(fname):
        with smart_open.open(url, "rb") as fin:
            with smart_open.open(fname, 'wb') as fout:
                while True:
                    buf = fin.read(io.DEFAULT_BUFFER_SIZE)
                    if not buf:
                        break
                    fout.write(buf)
                         
    with tarfile.open(fname, mode='r:gz') as tar:
        # Ignore directory entries, as well as files like README, etc.
        files = [
            m for m in tar.getmembers()
            if m.isfile() and re.search(r'nipstxt/nips\d+/\d+\.txt', m.name)
        ]
        for member in sorted(files, key=lambda x: x.name):
            member_bytes = tar.extractfile(member).read()
            yield member_bytes.decode('utf-8', errors='replace')

docs = list(extract_documents())

So we have a list of 1740 documents, where each document is a Unicode string. 
If you're thinking about using your own corpus, then you need to make sure
that it's in the same format (list of Unicode strings) before proceeding
with the rest of this tutorial.




In [ ]:
print(len(docs))
print(docs[0][:500])

In [4]:
import pandas as pd
twitter_df = pd.read_csv('../Data/all_tweets_clean.csv')

In [5]:
twitter_df = twitter_df['text']

In [6]:
twitter_df = twitter_df.astype('str')

need to covert str to 'utf-8' if there is an issue down the line

Pre-process and vectorize the documents
---------------------------------------

As part of preprocessing, we will:

* Tokenize (split the documents into tokens).
* Lemmatize the tokens.
* Compute bigrams.
* Compute a bag-of-words representation of the data.

First we tokenize the text using a regular expression tokenizer from NLTK. We
remove numeric tokens and tokens that are only a single character, as they
don't tend to be useful, and the dataset contains a lot of them.

.. Important::

   This tutorial uses the nltk library for preprocessing, although you can
   replace it with something else if you want.




In [ ]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [8]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(twitter_df)):
    twitter_df[idx] = twitter_df[idx].lower()  # Convert to lowercase.
    twitter_df[idx] = tokenizer.tokenize(twitter_df[idx])  # Split into words.

We use the WordNet lemmatizer from NLTK. A lemmatizer is preferred over a
stemmer in this case because it produces more readable words. Output that is
easy to read is very desirable in topic modelling.




In [ ]:
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [9]:
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
twitter_df = [[lemmatizer.lemmatize(token) for token in doc] for doc in twitter_df]

We find bigrams in the documents. Bigrams are sets of two adjacent words.
Using bigrams we can get phrases like "machine_learning" in our output
(spaces are replaced with underscores); without bigrams we would only get
"machine" and "learning".

Note that in the code below, we find bigrams and then add them to the
original data, because we would like to keep the words "machine" and
"learning" as well as the bigram "machine_learning".

.. Important::
    Computing n-grams of large dataset can be very computationally
    and memory intensive.




In [ ]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [10]:
# Compute bigrams.
from gensim.models import Phrases

bigram = Phrases(twitter_df, min_count=20)
for idx in range(len(twitter_df)):
    for token in bigram[twitter_df[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            twitter_df[idx].append(token)

2020-02-13 18:26:16,331 : INFO : 'pattern' package not found; tag filters are not available for English
2020-02-13 18:26:16,337 : INFO : collecting all words and their counts
2020-02-13 18:26:16,337 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2020-02-13 18:26:16,721 : INFO : PROGRESS: at sentence #10000, processed 207598 words and 112282 word types
2020-02-13 18:26:16,943 : INFO : PROGRESS: at sentence #20000, processed 333871 words and 164588 word types
2020-02-13 18:26:17,263 : INFO : PROGRESS: at sentence #30000, processed 507407 words and 228988 word types
2020-02-13 18:26:17,677 : INFO : PROGRESS: at sentence #40000, processed 733146 words and 304402 word types
2020-02-13 18:26:18,053 : INFO : PROGRESS: at sentence #50000, processed 931860 words and 361454 word types
2020-02-13 18:26:18,426 : INFO : PROGRESS: at sentence #60000, processed 1132385 words and 413856 word types
2020-02-13 18:26:18,729 : INFO : collected 453035 word types from a corpus of 1288

We remove rare words and common words based on their *document frequency*.
Below we remove words that appear in less than 20 documents or in more than
50% of the documents. Consider trying to remove words only based on their
frequency, or maybe combining that with this approach.




In [ ]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

In [11]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

dictionary = Dictionary(twitter_df)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

2020-02-13 18:26:41,940 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-02-13 18:26:42,339 : INFO : adding document #10000 to Dictionary(19086 unique tokens: ['above', 'depression', 'dorian', 'ewuls', 'going']...)
2020-02-13 18:26:42,630 : INFO : adding document #20000 to Dictionary(27051 unique tokens: ['above', 'depression', 'dorian', 'ewuls', 'going']...)
2020-02-13 18:26:43,001 : INFO : adding document #30000 to Dictionary(35543 unique tokens: ['above', 'depression', 'dorian', 'ewuls', 'going']...)
2020-02-13 18:26:43,473 : INFO : adding document #40000 to Dictionary(44642 unique tokens: ['above', 'depression', 'dorian', 'ewuls', 'going']...)
2020-02-13 18:26:43,904 : INFO : adding document #50000 to Dictionary(51719 unique tokens: ['above', 'depression', 'dorian', 'ewuls', 'going']...)
2020-02-13 18:26:44,323 : INFO : adding document #60000 to Dictionary(58734 unique tokens: ['above', 'depression', 'dorian', 'ewuls', 'going']...)
2020-02-13 18:26:44,646 : INFO 

Finally, we transform the documents to a vectorized form. We simply compute
the frequency of each word, including the bigrams.




In [ ]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [12]:
corpus = [dictionary.doc2bow(doc) for doc in twitter_df]

Let's see how many tokens and documents we have to train on.




In [ ]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

In [13]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 4852
Number of documents: 68216


Training
--------

We are ready to train the LDA model. We will first discuss how to set some of
the training parameters.

First of all, the elephant in the room: how many topics do I need? There is
really no easy answer for this, it will depend on both your data and your
application. I have used 10 topics here because I wanted to have a few topics
that I could interpret and "label", and because that turned out to give me
reasonably good results. You might not need to interpret all your topics, so
you could use a large number of topics, for example 100.

``chunksize`` controls how many documents are processed at a time in the
training algorithm. Increasing chunksize will speed up training, at least as
long as the chunk of documents easily fit into memory. I've set ``chunksize =
2000``, which is more than the amount of documents, so I process all the
data in one go. Chunksize can however influence the quality of the model, as
discussed in Hoffman and co-authors [2], but the difference was not
substantial in this case.

``passes`` controls how often we train the model on the entire corpus.
Another word for passes might be "epochs". ``iterations`` is somewhat
technical, but essentially it controls how often we repeat a particular loop
over each document. It is important to set the number of "passes" and
"iterations" high enough.

I suggest the following way to choose iterations and passes. First, enable
logging (as described in many Gensim tutorials), and set ``eval_every = 1``
in ``LdaModel``. When training the model look for a line in the log that
looks something like this::

   2016-06-21 15:40:06,753 - gensim.models.ldamodel - DEBUG - 68/1566 documents converged within 400 iterations

If you set ``passes = 20`` you will see this line 20 times. Make sure that by
the final passes, most of the documents have converged. So you want to choose
both passes and iterations to be high enough for this to happen.

We set ``alpha = 'auto'`` and ``eta = 'auto'``. Again this is somewhat
technical, but essentially we are automatically learning two parameters in
the model that we usually would have to specify explicitly.




In [14]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2020-02-13 18:27:49,298 : INFO : using autotuned alpha, starting with [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
2020-02-13 18:27:49,300 : INFO : using serial LDA version on this node
2020-02-13 18:27:49,307 : INFO : running online (multi-pass) LDA training, 10 topics, 20 passes over the supplied corpus of 68216 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2020-02-13 18:27:49,307 : INFO : PROGRESS: pass 0, at document #2000/68216
2020-02-13 18:27:51,341 : INFO : optimized alpha [0.07166523, 0.08229983, 0.06995095, 0.07606682, 0.070500314, 0.07390742, 0.06942973, 0.07067217, 0.09583956, 0.07610783]
2020-02-13 18:27:51,344 : INFO : merging changes from 2000 documents into a model of 68216 documents
2020-02-13 18:27:51,349 : INFO : topic #6 (0.069): 0.055*"to" + 0.033*"the" + 0.020*"and" + 0.020*"in" + 0.017*"for" + 0.016*"a" + 0.014*"you" + 0.012*"we" + 0.009*"of" + 0.009*"is"

2020-02-13 18:27:57,107 : INFO : topic #8 (0.135): 0.053*"the" + 0.037*"to" + 0.031*"a" + 0.023*"of" + 0.021*"and" + 0.015*"for" + 0.015*"is" + 0.014*"i" + 0.013*"in" + 0.013*"it"
2020-02-13 18:27:57,108 : INFO : topic diff=0.219476, rho=0.408248
2020-02-13 18:27:57,113 : INFO : PROGRESS: pass 0, at document #14000/68216
2020-02-13 18:27:57,902 : INFO : optimized alpha [0.06673159, 0.09577594, 0.065407686, 0.07743328, 0.06530499, 0.07603171, 0.06702899, 0.0674561, 0.13692787, 0.08252755]
2020-02-13 18:27:57,905 : INFO : merging changes from 2000 documents into a model of 68216 documents
2020-02-13 18:27:57,909 : INFO : topic #4 (0.065): 0.043*"the" + 0.040*"to" + 0.029*"in" + 0.026*"help" + 0.020*"of" + 0.019*"by" + 0.018*"and" + 0.017*"a" + 0.016*"more" + 0.014*"need"
2020-02-13 18:27:57,910 : INFO : topic #2 (0.065): 0.089*"to" + 0.034*"the" + 0.033*"relief" + 0.032*"in" + 0.032*"help" + 0.026*"effort" + 0.015*"by" + 0.014*"and" + 0.014*"relief_effort" + 0.014*"with"
2020-02-13 18:27

2020-02-13 18:28:01,622 : INFO : topic #8 (0.155): 0.050*"the" + 0.039*"to" + 0.030*"a" + 0.020*"of" + 0.018*"is" + 0.018*"and" + 0.015*"in" + 0.013*"for" + 0.012*"it" + 0.011*"this"
2020-02-13 18:28:01,622 : INFO : topic diff=0.229907, rho=0.288675
2020-02-13 18:28:01,629 : INFO : PROGRESS: pass 0, at document #26000/68216
2020-02-13 18:28:02,506 : INFO : optimized alpha [0.068602264, 0.108514674, 0.066809386, 0.08152887, 0.06671219, 0.08344291, 0.07063141, 0.07040973, 0.16479994, 0.09245242]
2020-02-13 18:28:02,509 : INFO : merging changes from 2000 documents into a model of 68216 documents
2020-02-13 18:28:02,513 : INFO : topic #4 (0.067): 0.038*"to" + 0.034*"in" + 0.032*"the" + 0.029*"help" + 0.023*"of" + 0.022*"by" + 0.021*"are" + 0.019*"need" + 0.019*"and" + 0.016*"we"
2020-02-13 18:28:02,514 : INFO : topic #2 (0.067): 0.102*"to" + 0.038*"relief" + 0.033*"help" + 0.030*"in" + 0.024*"effort" + 0.024*"the" + 0.023*"for" + 0.019*"by" + 0.019*"those" + 0.017*"support"
2020-02-13 18:2

2020-02-13 18:28:06,898 : INFO : topic #8 (0.236): 0.053*"the" + 0.037*"to" + 0.032*"a" + 0.024*"and" + 0.024*"of" + 0.017*"in" + 0.017*"is" + 0.013*"this" + 0.011*"flood" + 0.011*"for"
2020-02-13 18:28:06,898 : INFO : topic diff=0.196420, rho=0.235702
2020-02-13 18:28:06,904 : INFO : PROGRESS: pass 0, at document #38000/68216
2020-02-13 18:28:07,724 : INFO : optimized alpha [0.07010491, 0.10522102, 0.07581063, 0.1033231, 0.07802055, 0.09798673, 0.08322953, 0.08283355, 0.2548803, 0.110829554]
2020-02-13 18:28:07,727 : INFO : merging changes from 2000 documents into a model of 68216 documents
2020-02-13 18:28:07,731 : INFO : topic #0 (0.070): 0.043*"a" + 0.039*"firefighter" + 0.025*"n" + 0.024*"de" + 0.018*"wind" + 0.018*"update" + 0.018*"y" + 0.015*"o" + 0.014*"la" + 0.013*"hurricane"
2020-02-13 18:28:07,732 : INFO : topic #2 (0.076): 0.094*"to" + 0.031*"help" + 0.028*"the" + 0.026*"by" + 0.026*"relief" + 0.025*"those" + 0.025*"california" + 0.025*"in" + 0.025*"for" + 0.019*"effort"
20

2020-02-13 18:28:11,657 : INFO : topic #8 (0.336): 0.063*"the" + 0.043*"fire" + 0.034*"to" + 0.031*"a" + 0.024*"of" + 0.023*"and" + 0.018*"in" + 0.018*"is" + 0.014*"this" + 0.011*"from"
2020-02-13 18:28:11,658 : INFO : topic diff=0.182592, rho=0.204124
2020-02-13 18:28:11,664 : INFO : PROGRESS: pass 0, at document #50000/68216
2020-02-13 18:28:12,414 : INFO : optimized alpha [0.08187267, 0.10603597, 0.090115555, 0.122035496, 0.08183326, 0.10221283, 0.09071685, 0.12311832, 0.35598642, 0.1231389]
2020-02-13 18:28:12,417 : INFO : merging changes from 2000 documents into a model of 68216 documents
2020-02-13 18:28:12,421 : INFO : topic #0 (0.082): 0.051*"firefighter" + 0.027*"a" + 0.027*"wind" + 0.023*"de" + 0.022*"update" + 0.022*"n" + 0.017*"santa" + 0.015*"bay" + 0.015*"forest" + 0.014*"northwest"
2020-02-13 18:28:12,422 : INFO : topic #4 (0.082): 0.037*"the" + 0.034*"to" + 0.028*"by" + 0.023*"in" + 0.022*"help" + 0.021*"and" + 0.021*"are" + 0.020*"fire" + 0.018*"of" + 0.017*"more"
2020

...

2020-02-13 18:32:24,277 : INFO : topic #5 (1.161): 0.066*"it" + 0.061*"i" + 0.037*"my" + 0.026*"not" + 0.025*"that" + 0.025*"about" + 0.025*"so" + 0.025*"wa" + 0.024*"s" + 0.022*"but"
2020-02-13 18:32:24,278 : INFO : topic #3 (1.235): 0.066*"for" + 0.051*"all" + 0.047*"our" + 0.047*"thomas" + 0.043*"thomas_fire" + 0.024*"to" + 0.021*"who" + 0.019*"their" + 0.018*"are" + 0.018*"thank"
2020-02-13 18:32:24,278 : INFO : topic #8 (6.049): 0.088*"the" + 0.056*"to" + 0.046*"in" + 0.043*"a" + 0.040*"of" + 0.037*"and" + 0.026*"is" + 0.019*"on" + 0.018*"fire" + 0.017*"for"
2020-02-13 18:32:24,279 : INFO : topic diff=0.060482, rho=0.135947
2020-02-13 18:32:24,286 : INFO : PROGRESS: pass 19, at document #20000/68216
2020-02-13 18:32:24,564 : INFO : optimized alpha [0.3623927, 0.4875639, 0.36860234, 1.2151042, 0.68250954, 1.168053, 0.62013173, 0.57230496, 6.0199714, 0.56757766]
2020-02-13 18:32:24,567 : INFO : merging changes from 2000 documents into a model of 68216 documents
2020-02-13 18:32:24,5

2020-02-13 18:32:26,176 : INFO : topic #3 (1.180): 0.074*"for" + 0.057*"our" + 0.054*"all" + 0.025*"to" + 0.022*"their" + 0.022*"thomas" + 0.021*"who" + 0.021*"are" + 0.020*"thomas_fire" + 0.019*"family"
2020-02-13 18:32:26,177 : INFO : topic #5 (1.248): 0.066*"it" + 0.063*"i" + 0.039*"my" + 0.026*"not" + 0.025*"that" + 0.025*"but" + 0.024*"about" + 0.024*"so" + 0.023*"s" + 0.020*"what"
2020-02-13 18:32:26,178 : INFO : topic #8 (6.168): 0.081*"the" + 0.058*"to" + 0.043*"a" + 0.043*"in" + 0.040*"of" + 0.037*"and" + 0.026*"is" + 0.019*"on" + 0.018*"for" + 0.014*"from"
2020-02-13 18:32:26,179 : INFO : topic diff=0.041564, rho=0.135947
2020-02-13 18:32:26,185 : INFO : PROGRESS: pass 19, at document #32000/68216
2020-02-13 18:32:26,492 : INFO : optimized alpha [0.34460467, 0.49627912, 0.35538408, 1.1958156, 0.7014644, 1.2602954, 0.6754336, 0.5264435, 6.239337, 0.6093698]
2020-02-13 18:32:26,495 : INFO : merging changes from 2000 documents into a model of 68216 documents
2020-02-13 18:32:26,

2020-02-13 18:32:28,132 : INFO : topic #3 (1.204): 0.066*"for" + 0.053*"our" + 0.051*"all" + 0.025*"to" + 0.023*"their" + 0.023*"who" + 0.021*"are" + 0.021*"home" + 0.020*"thank" + 0.018*"thank_you"
2020-02-13 18:32:28,133 : INFO : topic #5 (1.242): 0.069*"it" + 0.063*"i" + 0.036*"my" + 0.029*"not" + 0.028*"that" + 0.026*"s" + 0.026*"so" + 0.026*"but" + 0.024*"wa" + 0.023*"about"
2020-02-13 18:32:28,134 : INFO : topic #8 (6.296): 0.086*"the" + 0.057*"to" + 0.042*"in" + 0.041*"of" + 0.040*"a" + 0.040*"and" + 0.025*"is" + 0.018*"on" + 0.016*"for" + 0.014*"from"
2020-02-13 18:32:28,135 : INFO : topic diff=0.054851, rho=0.135947
2020-02-13 18:32:28,142 : INFO : PROGRESS: pass 19, at document #44000/68216
2020-02-13 18:32:28,453 : INFO : optimized alpha [0.35814947, 0.46611863, 0.34886345, 1.2058007, 0.6929545, 1.2370164, 0.6953451, 0.6029046, 6.2828984, 0.60099983]
2020-02-13 18:32:28,456 : INFO : merging changes from 2000 documents into a model of 68216 documents
2020-02-13 18:32:28,460 :

2020-02-13 18:32:30,022 : INFO : topic #2 (0.381): 0.142*"california" + 0.087*"victim" + 0.062*"link" + 0.038*"click" + 0.035*"northern" + 0.032*"link_victim" + 0.032*"click_bio" + 0.026*"northern_california" + 0.025*"effort" + 0.022*"k"
2020-02-13 18:32:30,023 : INFO : topic #5 (1.132): 0.071*"it" + 0.064*"i" + 0.034*"my" + 0.030*"wa" + 0.029*"s" + 0.028*"that" + 0.027*"not" + 0.027*"so" + 0.025*"about" + 0.025*"but"
2020-02-13 18:32:30,023 : INFO : topic #3 (1.144): 0.061*"for" + 0.048*"our" + 0.044*"all" + 0.027*"their" + 0.024*"home" + 0.024*"who" + 0.023*"to" + 0.021*"are" + 0.021*"thank" + 0.019*"thank_you"
2020-02-13 18:32:30,024 : INFO : topic #8 (6.201): 0.088*"the" + 0.049*"to" + 0.042*"in" + 0.039*"and" + 0.038*"of" + 0.037*"a" + 0.037*"fire" + 0.024*"is" + 0.019*"carr" + 0.017*"carr_fire"
2020-02-13 18:32:30,025 : INFO : topic diff=0.026311, rho=0.135947
2020-02-13 18:32:30,031 : INFO : PROGRESS: pass 19, at document #56000/68216
2020-02-13 18:32:30,323 : INFO : optimized a

2020-02-13 18:32:31,905 : INFO : merging changes from 2000 documents into a model of 68216 documents
2020-02-13 18:32:31,909 : INFO : topic #0 (0.377): 0.085*"firefighter" + 0.049*"wind" + 0.031*"southern" + 0.028*"southern_california" + 0.026*"santa" + 0.024*"n" + 0.023*"la" + 0.022*"de" + 0.017*"national" + 0.017*"park"
2020-02-13 18:32:31,910 : INFO : topic #2 (0.398): 0.211*"california" + 0.086*"victim" + 0.050*"link" + 0.033*"click" + 0.026*"link_victim" + 0.025*"click_bio" + 0.023*"northern" + 0.022*"effort" + 0.020*"k" + 0.016*"northern_california"
2020-02-13 18:32:31,910 : INFO : topic #5 (1.072): 0.070*"it" + 0.063*"i" + 0.035*"my" + 0.034*"wa" + 0.031*"so" + 0.030*"s" + 0.029*"that" + 0.026*"not" + 0.026*"but" + 0.024*"about"
2020-02-13 18:32:31,911 : INFO : topic #3 (1.319): 0.092*"thomas" + 0.089*"thomas_fire" + 0.053*"for" + 0.036*"our" + 0.034*"all" + 0.020*"home" + 0.020*"to" + 0.018*"their" + 0.018*"who" + 0.016*"thank"
2020-02-13 18:32:31,912 : INFO : topic #8 (6.264):

We can compute the topic coherence of each topic. Below we display the
average topic coherence and print the topics in order of topic coherence.

Note that we use the "Umass" topic coherence measure here (see
:py:func:`gensim.models.ldamodel.LdaModel.top_topics`), Gensim has recently
obtained an implementation of the "AKSW" topic coherence measure (see
accompanying blog post, http://rare-technologies.com/what-is-topic-coherence/).

If you are familiar with the subject of the articles in this dataset, you can
see that the topics below make a lot of sense. However, they are not without
flaws. We can see that there is substantial overlap between some topics,
others are hard to interpret, and most of them have at least some terms that
seem out of place. If you were able to do better, feel free to share your
methods on the blog at http://rare-technologies.com/lda-training-tips/ !




In [15]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2020-02-13 18:32:39,117 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2020-02-13 18:32:39,129 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2020-02-13 18:32:39,142 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2020-02-13 18:32:39,154 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2020-02-13 18:32:39,167 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2020-02-13 18:32:39,181 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2020-02-13 18:32:39,194 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2020-02-13 18:32:39,210 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2020-02-13 18:32:39,226 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2020-02-13 18:32:39,240 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2020-02-13 18:32:39,253 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2020-02-13 18:32:39

Average topic coherence: -5.7352.
[([(0.09618504, 'the'),
   (0.05551272, 'fire'),
   (0.047239292, 'in'),
   (0.04481858, 'to'),
   (0.03749885, 'of'),
   (0.03701674, 'and'),
   (0.03691665, 'a'),
   (0.02282366, 'is'),
   (0.017537164, 'on'),
   (0.014384335, 'from'),
   (0.013518414, 'for'),
   (0.012164928, 'at'),
   (0.011635928, 'this'),
   (0.0098210145, 'carr'),
   (0.009505388, 'have'),
   (0.009394532, 'are'),
   (0.009128854, 'with'),
   (0.00896116, 'carr_fire'),
   (0.007593843, 'we'),
   (0.007506459, 'now')],
  -1.993767965376516),
 ([(0.0749711, 'it'),
   (0.061521765, 'i'),
   (0.03417259, 'my'),
   (0.033552885, 'wa'),
   (0.031789467, 'so'),
   (0.031134306, 's'),
   (0.029735722, 'that'),
   (0.02545381, 'but'),
   (0.02429822, 'not'),
   (0.02267734, 'just'),
   (0.0224117, 'about'),
   (0.018590387, 'what'),
   (0.017041462, 'they'),
   (0.0152372215, 't'),
   (0.014512878, 'like'),
   (0.014031343, 'when'),
   (0.0117358705, 'his'),
   (0.0101135075, 'me'),
   (

In [ ]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Things to experiment with
-------------------------

* ``no_above`` and ``no_below`` parameters in ``filter_extremes`` method.
* Adding trigrams or even higher order n-grams.
* Consider whether using a hold-out set or cross-validation is the way to go for you.
* Try other datasets.

Where to go from here
---------------------

* Check out a RaRe blog post on the AKSW topic coherence measure (http://rare-technologies.com/what-is-topic-coherence/).
* pyLDAvis (https://pyldavis.readthedocs.io/en/latest/index.html).
* Read some more Gensim tutorials (https://github.com/RaRe-Technologies/gensim/blob/develop/tutorials.md#tutorials).
* If you haven't already, read [1] and [2] (see references).

References
----------

1. "Latent Dirichlet Allocation", Blei et al. 2003.
2. "Online Learning for Latent Dirichlet Allocation", Hoffman et al. 2010.


